# Multiclass Regression

Poisson regression is also a special case of the generalized linear model, where the random component is specified by the Poisson distribution.<br><br>
**Sections**
- [1.0 Synthetic Data & Model](#1.0-Synthetic-Data-&-Model)
- [2.0 Newton Raphson Algorithm](#2.0-Newton-Raphson-Algorithm)
- [3.0 NR Implementation](#3.0-Newton-Raphson-Implementation)
    - [3.1 Checking Convergence](#3.1-Checking-Convergence)
- [4.0 Variance of Estimators](#4.0-Variance-of-Estimators)
    - [4.1 Covariance of $L'(\theta)$](#4.1-Covariance-of-$L'(\theta)$)
    - [4.2 Covariance of $\hat\theta$](#4.2-Covariance-of-$\hat\theta$)

### 0. Importing Modules

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import bokeh
from bokeh.plotting import figure, show
from bokeh.models import tickers, ranges
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## 1.0 Synthetic Data & Model

$$ K = 4 $$

\begin{align}
\large
P(Y_i = j \hspace{1 mm} |  \hspace{1 mm} \beta , x_i) = \dfrac{ {\rm e}^{x_i \beta_j}}{1 + \Sigma_{j=1}^{K-1} {\rm e}^{x_i \beta_j}}, \hspace{3 mm} for \hspace{3 mm} j = 1,2,3
\end{align}

$$\large = \dfrac{ {\rm e}^{x_i \beta_j}}{1 + \Sigma_{j=1}^{3} {\rm e}^{x_i \beta_j}}$$

\begin{align}
\large
P(Y_i = 4 \hspace{1 mm} |  \hspace{1 mm} \beta , x_i) = \dfrac{ {\rm e}^{x_i \beta_j}}{1 + \Sigma_{j=1}^{3} {\rm e}^{x_i \beta_j}}
\end{align}

(1) Write down the log-likelihood function

 $$\large L (\beta) = \prod_{i=1}^{n} p(y_i \hspace{1 mm} |  \hspace{1 mm} x) $$

$$\large log (L (\beta)) = \Sigma_{j=1}^{K-1}\hspace{1 mm}\beta_j \cdot \hspace{3 mm} \Sigma_{y_i=j}\hspace{1 mm} x_i - \hspace{3 mm}
\Sigma_{i=1}^{n}\hspace{1 mm} log \hspace{1 mm} (1 + \Sigma_{j=1}^{K-1} {\rm e}^{x_i \beta_j})$$

$$\large log (L (\beta)) = \Sigma_{j=1}^{K-1}\hspace{1 mm}\beta_j \cdot \hspace{3 mm} \Sigma_{y_i=j}\hspace{1 mm} x_i - \hspace{3 mm}
\Sigma_{i=1}^{n}\hspace{1 mm} log \hspace{1 mm} (1 + \Sigma_{j=1}^{K-1} {\rm e}^{x_i \beta_j})$$

$$\large log (L (\beta)) = \Sigma_{j=1}^{3}\hspace{1 mm}\beta_j \cdot \hspace{3 mm} \Sigma_{y_i=j}\hspace{1 mm} x_i - \hspace{3 mm}
\Sigma_{i=1}^{50}\hspace{1 mm} log \hspace{1 mm} (1 + \Sigma_{j=1}^{3} {\rm e}^{x_i \beta_j})$$

(2) Expressions of the partial derivatives

$$\large \dfrac{\partial L}{\partial \beta_j} \text{      and      }  
\dfrac{\partial ^2L}{\partial \beta_j \partial \beta_k} $$

$$\large \dfrac{\partial L}{\partial \beta_j} = \Sigma_{y_i=j}\hspace{1 mm} x_i 
- \Sigma_{i=1}^n \dfrac{ x_i{\rm e}^{x_i \beta_j}}{1 + \Sigma_{j=1}^{K-1} {\rm e}^{x_i \beta_j}}$$

$$\large \dfrac{\partial ^2L}{\partial \beta_j \partial \beta_k} = 
-1(j=k) \hspace{3 mm}
\Sigma_{i=1}^n \dfrac{ x_i^2{\rm e}^{x_i \beta_j}}{1 + \Sigma_{j=1}^{K-1} {\rm e}^{x_i \beta_j}}
+ \Sigma_{i=1}^n \dfrac{ x_i^2{\rm e}^{x_i (\beta_j + \beta_k)}}{(1 + \Sigma_{j=1}^{K-1} {\rm e}^{x_i \beta_j})^2}$$

## 2.0 Newton Raphson Algorithm

In [67]:
def get_L1_vector(x, yi, Bj):
    """
    Computes partial derivatives dL/dBj.

        Assumes Bj is a row vector with K−1 entries and X is a column array.

    Args:
        xi (np.ndarray): Column vector or N observation x M features
            matrix
        yi (np.ndarray): Column vector with categorical data.
        Bj (np.ndarray): Row vector

    """
    dL_dBj = []

    for category, bj in enumerate(Bj, start = 1):

        term_1 = np.sum(x[yi == category])
        denominator = np.ones(shape = x.shape)        
        for i, xi in enumerate(x):
            denominator[i] = (1 + np.sum(np.exp(xi * Bj))) # Vector Scaling
    
        numerator = x * np.exp(x * bj)
        dL_dBj.append(term_1 - np.sum(numerator/denominator))

    return dL_dBj

def get_Lprime2_matrix(x, K, Bj):
    """
    Computes partial second derivatives d2L/dBj dBk.

    Assumes Bj is a row vector with K−1 entries and X is a column array.

    Args:
        xi (np.ndarray): Column vector or N observation x M features
            matrix
        K (int): Number of categories or discrete values y can take
            from 1 to K.
        Bj (np.ndarray): Row vector with regression parameters.

    """
 
    l_prime2 = np.zeros(shape = (K-1, K-1)) # Matrix L2 is (j,k)

    # Approach: 
    #   Since x is a column vector and Bj is a row, vectorized operations make
    #   more sense. 
    #   The only explicit iteration 1:n is for the denominator.
    denominator = np.ones(shape = x.shape)

    for i, xi in enumerate(x):
        denominator[i] = (1 + np.sum(np.exp(xi * Bj))) # Vector Scaling
    
    # Note: symmetric matrix, we are esimating K-1 parameters
    for j in range(K-1):
        for k in range(0, K-1): #

            if j == k:
                f = -1
            else:
                f = 0

            l_prime2[j, k] = f*np.sum(x**2*np.exp(x * Bj[j])/denominator) + \
                    np.sum(x**2*np.exp(x * (Bj[j] + Bj[k]))/(denominator**2))

    return l_prime2

def newton_raphson(xArr, yArr, b_0, tolerance = 0.00001):
    """
    Performs Newton-Raphson root finding.
    
    Args:
        xArr (np.ndarray): Column array with x values.
        yArr (np.ndarray): Column array with y values (discrete).
        b_0 (float): Initial guess for regression parameters.
        tolerance (float): Stops iteration when difference between iterations
            is within tolerance.
    """

    k = len(b_0) + 1
    difference = tolerance * 5
    
    beta_iter = [b_0]
    while abs(difference) > tolerance:
        
        L_1 = get_L1_vector(xArr, yArr, b_0)
        L_2 = get_Lprime2_matrix(xArr, k, b_0)
        beta_1 = b_0 - np.linalg.solve(L_2, L_1)

        # Calculate difference and update iteration state
        difference = max(abs(np.array(beta_1) - np.array(b_0)))
        b_0 = beta_1
        beta_iter.append(b_0)
    
    return beta_1, beta_iter

## 3.0 Newton Raphson Implementation

(3) Using the partial derivatives just found, write and run a Newton{Raphson
algorithm to obtain the maximum likelihood estimator $\hat \beta$. <br>
State the algorithm and the final result.

In [62]:
df = pd.read_csv('data_1.csv')
df.head()

,y,x
0,2,0.208561
1,4,0.002906
2,2,0.392529
3,2,0.836454
4,3,0.465919


In [82]:
xArr = df.x.values
yArr = df.y.values

# Run NR with different starting points
betas = [np.array([0.5, 0.5, 0.5]),
         np.array([1, 1, 1]),
         np.array([4, 4, 4]),
         np.array([-2, -2, -2]),
        ]

root = [] # Container for different starting points
for b_0 in betas:
    beta_1, _ = newton_raphson(xArr, yArr, b_0)
    root.append(beta_1)

In [83]:
df_nr = pd.DataFrame(root)
beta_hat = pd.Series(df_nr.mean().values, index = ['B0', 'B1', 'B2'], name = 'B-hats')
beta_hat

B0    0.940669
B1    1.769768
B2    2.585184
Name: B-hats, dtype: float64

## 4.0 Prediction at X values

(4) Find the predictive probabilities for y with a new predictor at x = $\bar x$.

In [92]:
x_bar = xArr.mean()

In [ ]:
def get_p_j_given_x(x, Bj):
    """Calculates P( y = j | x).

    Args:
        x: N x M features
        B: M features x K-1

    Returns:
        np.ndarray: N x K-1 matrix with the probabilities of each observation
        to be classified as a given category. 
    """
    numerator = np.exp(x @ Bj) # Returns N x K-1 Matrix
    # Note: It is critical to sum over the axis because it is
    # only within an observation that the probabilities must add up to 1.
    denominator = (1 + 
                    np.sum(np.exp(x @ Bj), axis = 1)).reshape(-1,1) # N Vector
    
    return numerator / denominator

def get_pK(x, Bj):
    """Calculates P( y = K | x).
    """    
    denominator = (1 + np.sum(np.exp(x @ Bj), axis = 1)).reshape(-1,1)
    return 1 / denominator

In [106]:
# Probabilities j = 1 through K-1
xi = np.array([x_bar]).reshape(1,1)
beta_hat = beta_hat.values.reshape(1,-1)

p_array_K_minus_one = get_p_j_given_x(xi, beta_hat)

# Probabilities j = K
p_K = 1 - np.sum(p_array_K_minus_one, axis = 1).reshape(-1,1)

# Same as doing
p_K_v2 = get_pK(xi, beta_hat)
assert max(abs(p_K - p_K_v2)) < 10**-15

# Full array
p_array = np.concatenate([p_array_K_minus_one, p_K], axis = 1)
p_array

array([[0.17724729, 0.27952478, 0.43751799, 0.10570995]])

(5) Hence, what would be the predicted outcome for $y$ at this $x$.

In [109]:
print('Predicted outcome for y =' ,p_array.argmax() + 1 )

Predicted outcome for y = 3


## Illustration

In [84]:
n = 10000
x_i = np.random.normal(0, 1, size = (n,1))
Bj = np.array([-0.2,0,0.2,0.4]).reshape(1,-1)

In [85]:
def get_p_j_given_x(x, Bj):
    """Calculates P( y = j | x).

    Args:
        x: N x M features
        B: M features x K-1

    Returns:
        np.ndarray: N x K-1 matrix with the probabilities of each observation
        to be classified as a given category. 
    """
    numerator = np.exp(x @ Bj) # Returns N x K-1 Matrix
    # Note: It is critical to sum over the axis because it is
    # only within an observation that the probabilities must add up to 1.
    denominator = (1 + 
                    np.sum(np.exp(x @ Bj), axis = 1)).reshape(-1,1) # N Vector
    
    return numerator / denominator

def get_pK(x, Bj):
    """Calculates P( y = K | x).
    """    
    denominator = (1 + np.sum(np.exp(x @ Bj), axis = 1)).reshape(-1,1)
    return 1 / denominator

In [86]:
# Probabilities j = 1 through K-1
p_array_K_minus_one = get_p_j_given_x(x_i, Bj)

# Probabilities j = K
p_K = 1 - np.sum(p_array_K_minus_one, axis = 1).reshape(-1,1)

# Same as doing
p_K_v2 = get_pK(x_i, Bj)
assert max(abs(p_K - p_K_v2)) < 10**-15

# Full array
p_array = np.concatenate([p_array_K_minus_one, p_K], axis = 1)

In [89]:
p_array[:10]

array([[0.17616689, 0.19232432, 0.20996366, 0.22922082, 0.19232432],
       [0.13607812, 0.17494139, 0.22490382, 0.28913528, 0.17494139],
       [0.21889326, 0.20489469, 0.19179136, 0.179526  , 0.20489469],
       [0.11166228, 0.16095443, 0.23200609, 0.33442276, 0.16095443],
       [0.1677732 , 0.18918154, 0.21332164, 0.24054209, 0.18918154],
       [0.32315545, 0.21684648, 0.14551015, 0.09764144, 0.21684648],
       [0.11546697, 0.16333367, 0.23104345, 0.32682225, 0.16333367],
       [0.22756757, 0.2068202 , 0.18796438, 0.17082764, 0.2068202 ],
       [0.17363428, 0.19140148, 0.21098672, 0.23257603, 0.19140148],
       [0.29486154, 0.21580231, 0.1579407 , 0.11559313, 0.21580231]])

Y values generation

In [87]:
y_i = []
for probabilities in p_array:
    y_random = np.random.choice(a = [1,2,3,4,5], 
                                size = 1,
                                p = probabilities)
    y_i.append(y_random[0])

## Generative classifier

In [32]:
from scipy.stats import norm

In [33]:
def get_prob_array(x, means, stds, Pj):
        
    normalizing_factor = 0
    pj_array = []
    
    # Note: The denominator is common to all
    for mean, std, pj in zip(means, stds, Pj):
        prob_N_times_pj = norm.pdf(x, loc = mean, scale = std) * pj
        pj_array.append(prob_N_times_pj)
        
        normalizing_factor += prob_N_times_pj

    pj_array = np.array(pj_array) / normalizing_factor
    return pj_array

In [34]:
means = [-0.26, -0.10, 0.12, 0.27, 0.10]
stds = [0.97, 1.00, 0.97, 0.99, 0.9749]
Pj = [0.2, 0.2, 0.2, 0.2, 0.2]

classes = []
xArr = np.arange(-3,3,0.05)
for x in xArr:    
    classes.append(get_prob_array(x, means, stds, Pj).argmax() + 1)

In [35]:
p = figure(toolbar_location= None, outline_line_color = 'black')
p.line(x = xArr, y = classes, line_width = 1, color = 'firebrick', legend_label="Data")
p.axis.axis_label = 'x'
p.yaxis.axis_label = 'Classification'
p.legend.border_line_color = "black"
p.legend.border_line_alpha = 1
p.legend.location = 'bottom_right'
show(p)    

### 2.0 Newton Raphson Algorithm

Estimating a and b via maximum likelihood. Helper function to get $J(\theta)$ matrix and $L'(\theta)$ vector

$$
J(a,b)=\begin{pmatrix}
    \dfrac{\partial^2 L}{\partial a^2}   & \dfrac{\partial^2 L}{\partial a \hspace{1mm} \partial b}\\
    \dfrac{\partial^2 L}{\partial a \hspace{1mm} \partial b}   & \dfrac{\partial^2 L}{\partial b^2}
\end{pmatrix} \\
\\
\text{}\\
L'(a,b) = (\dfrac{\partial L}{\partial a}, \dfrac{\partial L}{\partial b})
$$

In [ ]:
def get_J_and_Lprime(x_arr, y_arr, a, b, n):
    """
    Computes J matrix and L vector for a Poisson Distribution
    where lambda is model as:
    
    \begin{equation}
    \lambda  = {\rm e}^{a + bx}
    \end{equation}       
    """
    z = np.sum(x_arr*y_arr)

    # Get derivatives for NR
    dL_da = n*np.mean(y_arr) - np.exp(a)*np.sum(np.exp(b*x_arr))
    dL_db = z - np.exp(a)*np.sum(x_arr*np.exp(b*x_arr))

    # Second Partial Derivatives
    dL_da2 = - np.exp(a)*np.sum(np.exp(b*x_arr))
    dL_db2 = - np.exp(a)*np.sum((x_arr**2)*np.exp(b*x_arr))
    dL_dadb = - np.exp(a)*np.sum(x_arr*np.exp(b*x_arr))
    
    J = np.array([[dL_da2,  dL_dadb],
              [dL_dadb, dL_db2]])
    
    L_prime = np.array([[dL_da],[dL_db]])
    return J, L_prime

Compute $J(\theta)$ matrix and $L'(\theta)$ vector for true parameters

In [ ]:
J, L_prime = get_J_and_Lprime(x_i, y_i, a, b, n)
print("J matrix:\n", J)
print("\nL_prime vector:\n", L_prime)

Algorithm function

In [ ]:
def newton_n_iter(x, y, a_o, b_o,  tolerance = 0.00001, output_message = False):
    """
    Performs Newton-Raphson for a definite number of iteration
    Args:
        guess (float): initial value for parameter
        tolerance (float): tolerance
    
    """
    #Initialize
    a = [a_o]
    b = [b_o]
    difference = tolerance * 5 # Enter Loop
    iter_number = 0
    status_message = 'Starting with Guess = ' + str(a_o) + "," + str(b_o) + '\n'

    while abs(difference) > tolerance:
    
        J, L = get_J_and_Lprime(x_i, y_i, a_o, b_o, n)
   
        a_1 ,b_1 = np.array([[a_o],[b_o]]) - np.linalg.inv(J) @ L
        a.append(a_1[0])
        b.append(b_1[0])
        
        # calculate difference and update iteration state
        difference = max(a_1[0] - a_o, b_1[0] - b_o)
        a_o, b_o = a_1[0], b_1[0]
    
        iter_number += 1
        status_message += 'Iteration #' + str(iter_number) + ':= ' + str(a_1[0]) + "," + str(b_1[0])+ '\n'
        
    status_message += 'Total No. of Iterations = '  +  str(iter_number)
    
    if output_message:
        return a_o, b_o, status_message, a, b
    return a_o, b_o, a, b

### 3.0 Newton Raphson Implementation

In [ ]:
(a_hat, b_hat, status_message, a_array, b_array) = newton_n_iter(x_i, y_i, 1.6, 2, output_message = True)
print(status_message)

#### 3.1 Checking Convergence

In [ ]:
# Check that the derivatives of the parameters in last iteration are about zero
J, L_prime = get_J_and_Lprime(x_i, y_i, a_hat, b_hat, n)
print("\nL vector:\n", L_prime)

In [ ]:
# Plotting
p = figure(toolbar_location= None, outline_line_color = 'black')
p.plot_width = 350
p.plot_height = 350
p.line(x = range(len(a_array)), y = a_array, line_width = 2, line_color = 'firebrick', legend_label="a")
p.line(x = range(len(a_array)), y = b_array, line_width = 2, line_color = 'green', legend_label="b")
p.axis.axis_label = 'Iteration #'
p.yaxis.axis_label = 'a and b'
p.legend.border_line_color = "black"
p.legend.border_line_alpha = 1
p.legend.label_text_color = 'black'
p.legend.location = 'top_right'
show(p)  

### 4.0 Variance of Estimators

In [ ]:
print(a_hat)
print(b_hat)

#### 4.1 Covariance of $L'(\theta)$

$$
\large
Cov L'(a,b)=\begin{pmatrix}
    Var(\dfrac{\partial L}{\partial a})   & Cov(\dfrac{\partial L}{\partial a}, \dfrac{\partial L}{\partial b})\\
    Cov(\dfrac{\partial L}{\partial a}, \dfrac{\partial L}{\partial b})   & Var(\dfrac{\partial L}{\partial b})\\
    \end{pmatrix}  = \Omega 
     =\begin{pmatrix}     
     \Sigma_{i=1}^{n}\lambda_i   & \Sigma_{i=1}^{n}x_i\lambda_i\\
     \Sigma_{i=1}^{n}x_i\lambda_i   & \Sigma_{i=1}^{n}x_i^2\lambda_i 
\end{pmatrix} = \Omega
\\
\large
$$

In [ ]:
def get_COV_L_prime(x_arr, y_arr, a, b, n):
    """
    Computes the covariance matrix of L'   
    """
    lambda_i =  np.exp(x_arr + b*x_arr)
    # Get derivatives for NR
    dL_da = np.sum(lambda_i)
    dL_db = np.sum((x_arr**2) * lambda_i)
    dL_dadb = np.sum(x_arr * lambda_i)
    
    COV_L = np.array([[dL_da,  dL_dadb],
                      [dL_dadb, dL_db]])
    
    return COV_L

COV_L_prime = get_COV_L_prime(x_i, y_i, a_hat, b_hat, n)
COV_L_prime

#### 4.2 Covariance of $\hat\theta$

$$
\large
Cov\hspace{1mm}\hat\theta = J^{-1}(\theta)\hspace{2mm}CovL'(\theta)\hspace{2mm}J^{-1}(\theta)
\\
\large
Cov \hat\theta = \begin{pmatrix}
    Var(a)   & Cov(a,b)\\
    Cov(a,b)  & Var(b)\\
    \end{pmatrix}
$$

In [ ]:
COV_theta_hat = np.linalg.inv(J) @ COV_L_prime @ np.linalg.inv(J)
COV_theta_hat